In [ ]:
import numpy as np
from IPython.display import Audio
from scipy.signal import sawtooth

def generate_triad(root, inversion):
    # Ensure the root is within the range of a single octave
    root = root % 12

    # Define the intervals for a major triad
    major_third = (root + 4) % 12
    perfect_fifth = (root + 7) % 12
    
    if inversion == 1:
        root, major_third, perfect_fifth = (major_third + 12)%12, (perfect_fifth + 12)%12, root
        
    elif inversion == 2:
        root, major_third, perfect_fifth = (perfect_fifth + 12)%12, (root + 12)%12, (major_third + 12)%12

    # Return the triad as a tuple of three notes
    return root, major_third, perfect_fifth

# Example: Generate a C major triad (C, E, G)
#c_major_triad = generate_triad(0)
#print("C Major Triad:", c_major_triad)


# Function to generate a sine wave for a given frequency and duration
def generate_sine_wave_with_envelope(frequency, duration, sampling_rate=44100):
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)

    # Sine wave
    wave = 0.5 * np.sin(2 * np.pi * frequency * t)

    # Envelope (ADSR - Attack, Decay, Sustain, Release)
    attack_time = 0.05  # seconds
    decay_time = 0.1   # seconds
    release_time = 0.2  # seconds

    # Introduce a short overlap between attack and decay for smoother transitions
    overlap_time = 0.02  # seconds

    # Attack
    attack = np.linspace(0, 1, int(attack_time * sampling_rate))

    # Decay with overlap
    decay = np.linspace(1, 0.8, int(decay_time * sampling_rate))
    decay[:int(overlap_time * sampling_rate)] *= np.linspace(0, 1, int(overlap_time * sampling_rate))

    # Sustain
    sustain = np.full(int((duration - (attack_time + decay_time + release_time)) * sampling_rate), 0.8)

    # Release
    release = np.linspace(0.8, 0, int(release_time * sampling_rate))

    envelope = np.concatenate([attack, decay, sustain, release])

    # Ensure the envelope length matches the waveform length
    envelope = np.resize(envelope, len(t))

    return wave * envelope

def generate_piano_wave_with_envelope(frequency, duration, sampling_rate=44100):
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)

    # Sine wave
    sine_wave = 0.5 * np.sin(2 * np.pi * frequency * t)

    # Triangle wave
    triangle_wave = 0.2 * np.abs(sawtooth(2 * np.pi * frequency * t, 0.5))

    # Combine sine and triangle waves
    wave = 0.7 * sine_wave + 0.3 * triangle_wave

    # Envelope (ADSR - Attack, Decay, Sustain, Release)
    attack_time = 0.05  # seconds
    decay_time = 0.1   # seconds
    release_time = 0.2  # seconds

    # Introduce a short overlap between attack and decay for smoother transitions
    overlap_time = 0.02  # seconds

    # Attack
    attack = np.linspace(0, 1, int(attack_time * sampling_rate))

    # Decay with overlap
    decay = np.linspace(1, 0.8, int(decay_time * sampling_rate))
    decay[:int(overlap_time * sampling_rate)] *= np.linspace(0, 1, int(overlap_time * sampling_rate))

    # Sustain
    sustain = np.full(int((duration - (attack_time + decay_time + release_time)) * sampling_rate), 0.8)

    # Release
    release = np.linspace(0.8, 0, int(release_time * sampling_rate))

    envelope = np.concatenate([attack, decay, sustain, release])

    # Ensure the envelope length matches the waveform length
    envelope = np.resize(envelope, len(t))

    return wave * envelope


# Function to play a piano triad
def play_triad(root, inversion, duration=1.0):
    root, major_third, perfect_fifth = generate_triad(root, inversion)

    # Convert pitch classes to frequencies
    #frequencies = [440 * 2**(note/12) for note in [root, major_third, perfect_fifth]]
    if inversion == 1:
        frequencies = [440 * 2**(note/12) for note in [major_third, perfect_fifth, root]]
    elif inversion == 2:
        frequencies = [440 * 2**(note/12) for note in [perfect_fifth, root, major_third]]
    else:
        frequencies = [440 * 2**(note/12) for note in [root, major_third, perfect_fifth]]
        
    print("Frequencies : ", frequencies)

    # Generate sine waves for each note
    waveforms = [generate_piano_wave_with_envelope(freq, duration) for freq in frequencies]

    # Sum the waveforms to create the chord
    chord = np.sum(waveforms, axis=0)

    # Normalize the amplitude to avoid clipping
    chord = chord / np.max(np.abs(chord))

    # Play the chord
    display(Audio(chord, rate=44100))

# Example: Play a C major triad for 2 seconds
play_triad(0, inversion = 0, duration=2.0)
play_triad(0, inversion = 1, duration=2.0)
play_triad(0, inversion = 2, duration=2.0)
